In [ ]:
from shapely.geometry import Polygon
from geopandas import GeoDataFrame
import numpy as np
from pyproj import Transformer

lon=np.arange(-180, 180, 2)
lat=np.zeros_like(lon)+-88.

transformer = Transformer.from_crs("epsg:4326", "epsg:3031")
x_ph, y_ph = transformer.transform(lat, lon)

circle=Polygon(zip(x_ph, y_ph))
circle.buffer(-200)

gdf=GeoDataFrame({'geometry':[circle]})
gdf.set_crs('EPSG:3031')

gdf.to_file('/home/ben/git_repos/surfaceChange/masks/updates/S_polehole.geojson', driver='GeoJSON')




N.B.  After doing this, I still had to do this:

ogr2ogr -a_srs EPSG:3031 S_polehole_3031.geojson S_polehole.geojson

... then

gdal_rasterize -burn 0 S_polehole_3031.geojson bedmap2_thickness_gt_50_plus_sio_shelves_edited.tif


... Then, to see what tiles these masks hit:
gdal_rasterize -te -3040000 -3040000 3000000 3000000 -tr 80000 80000 -tap -init 0 -burn 1 -at 20210325_PIG_front.geojson 80km_update.tif

gdal_rasterize -burn 1 -at S_polehole_3031.geojson  80km_update.tif


In [ ]:
import pointCollection as pc
import matplotlib.pyplot as plt
import numpy as np

mask_80=pc.grid.data().from_geotif('/home/ben/git_repos/surfaceChange/masks/Antarctic/updates/80km_update.tif')
plt.figure()
mask_80.show()


In [ ]:
D_80=mask_80.as_points()
D_80=D_80[D_80.z==1]
np.c_[D_80.x, D_80.y]

In [ ]:
for xy in zip(D_80.x/1000, D_80.y/1000):
    print('E%d_N%d.h5'% xy)